<a href="https://colab.research.google.com/github/razaparvi/LLM-repos/blob/main/thousand_splendid_sun_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken qdrant-client langchain_openai  pypdf langchainhub  langchain_community langchain-groq

In [ ]:
!pip -q install langchain openai tiktoken qdrant-client langchain_openai  pypdf langchainhub  langchain_community langchain-groq langchain-huggingface

In [ ]:
 !pip command again.


In [ ]:
from langchain_groq import ChatGroq
from langchain import hub
import tiktoken
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from google.colab import userdata
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.document_loaders import  PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

In [ ]:
qdrant_url=userdata.get('QUADRANT_URL')
qdrant_api_key=userdata.get('Qdrant_API_key')
#openai_api_key=userdata.get('OPENAI_API_KEY')
groq_api_key=userdata.get('GROQ_API_key')

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader=PyPDFLoader("/content/a_thousand_splendid_sun.pdf")
documents=loader.load()

In [ ]:
documents[:5]

In [ ]:
len(documents)

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=2500,chunk_overlap=300)
texts=text_splitter.split_documents(documents)

In [ ]:
len(texts)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
qdrant=Qdrant.from_documents(
    texts,
    embedding=embeddings,
    url=qdrant_url,
    api_key=qdrant_api_key,
    prefer_grpc=True,
    collection_name="test_collection",

)

In [ ]:
qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

In [ ]:
qdrant_client=QdrantClient(
    url=qdrant_url,api_key=qdrant_api_key
)

In [ ]:
retriever=qdrant.as_retriever(search_kwargs={"k": 5})

In [ ]:
retriever=qdrant.as_retriever(search_kwargs={"k": 5})

query = "Who was Marium?"
docs=retriever.get_relevant_documents(query)

#for better print of the docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )


docs

pretty_print_docs(docs)

In [ ]:
retriever=qdrant.as_retriever(search_kwargs={"k": 10})

prompt=PromptTemplate(

template="""
# Your role
You are a brilliant expert at understanding the intent of the questioner and the crux of the question, and providing the most optimal answer  from the docs to the questioner's needs from the documents you are given.


# Instruction
Your task is to answer the question  using the following pieces of retrieved context delimited by XML tags.

<retrieved context>
Retrieved Context:
{context}
</retrieved context>


# Constraint
1. Think deeply and multiple times about the user's question\nUser's question:\n{question}\nYou must understand the intent of their question and provide the most appropriate answer.
- Ask yourself why to understand the context of the question and why the questioner asked it, reflect on it, and provide an appropriate response based on what you understand.
2. Choose the most relevant content(the key content that directly relates to the question) from the retrieved context and use it to generate an answer.
3. Generate a concise, logical answer. When generating the answer, Do Not just list your selections, But rearrange them in context so that they become paragraphs with a natural flow.
4. When you don't have retrieved context for the question or If you have a retrieved documents, but their content is irrelevant to the question, you should answer 'I can't find the answer to that question in the material I have'.
5. Use five sentences maximum. Keep the answer concise but logical/natural/in-depth.
6. At the end of the response provide metadata provided in the relevant docs , For example:"Metadata: page: 19, source: /content/OCR_RSCA/Analyse docs JVB + mails et convention FOOT INNOVATION.pdf'. Return Just the page and source


# Question:
{question}""",
    input_variables=["context","question"]
)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0,openai_api_key=openai_api_key)
groq_llm= ChatGroq(
    model="llma-3.1-70b-versatile",
    temperature=0,
    groq_api_key=groq_api_key
    max_retries=2,
)


def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        # Format the metadata into a string
        metadata_str = ', '.join(f"{key}: {value}" for key, value in doc.metadata.items())

        # Combine page content with its metadata
        doc_str = f"{doc.page_content}\nMetadata: {metadata_str}"

        # Append to the list of formatted documents
        formatted_docs.append(doc_str)

    # Join all formatted documents with double newlines
    return "\n\n".join(formatted_docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)